# <a id='toc1_'></a>[Supervised Learning - Regression](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Supervised Learning - Regression](#toc1_)    
- [Review dataset](#toc2_)    
  - [Review data types](#toc2_1_)    
  - [Review duplicates](#toc2_2_)    
  - [Review nulls](#toc2_3_)    
  - [Review unique categories](#toc2_4_)    
  - [Remove obvious racial bias](#toc2_5_)    
- [Train-test split](#toc3_)    
- [Linear Regression](#toc4_)    
  - [Review model coefficients](#toc4_1_)    
- [Ridge regression](#toc5_)    
  - [Lasso regression](#toc5_1_)    
  - [Retry regression without 'useless' features](#toc5_2_)    
- [Decision Tree Regression](#toc6_)    
- [KNN Regression](#toc7_)    
- [Regression evaluation metrics](#toc8_)    
  - [R/MSE](#toc8_1_)    
  - [$R^2$](#toc8_2_)    
- [References](#toc9_)    
- [Acknowledgements](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from dfply import *

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split

# <a id='toc2_'></a>[Review dataset](#toc0_)

> As explained in `sklearn.datasets.load_boston()`, Harrison and Rubenfield developed the feature B (result of the formula `1000(B_k - 0.63)^2k`) under the assumption that racial self-segregation had a positive impact on house prices. B then encodes systemic racism as a factor in house pricing. Thus, any models trained using this data that do not take special care to process B will learn to use mathematically encoded racism as a factor in house price prediction. [$^{[1]}$](https://fairlearn.org/main/user_guide/datasets/boston_housing_data.html)

In [3]:
from sklearn.datasets import fetch_openml

def load_boston_dataset():
    dataset = fetch_openml(name='boston', version=1)
    return dataset.data, dataset.target

X, y = load_boston_dataset()

c:\Users\sabin\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


> The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston MA. The following describes the dataset columns [$^{[2]}$](https://www.kaggle.com/code/prasadperera/the-boston-housing-dataset):

> CRIM - per capita crime rate by town  
> ZN - proportion of residential land zoned for lots over 25,000 sq.ft.  
> INDUS - proportion of non-retail business acres per town.  
> CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)  
> NOX - nitric oxides concentration (parts per 10 million)  
> RM - average number of rooms per dwelling  
> AGE - proportion of owner-occupied units built prior to 1940  
> DIS - weighted distances to five Boston employment centres  
> RAD - index of accessibility to radial highways  
> TAX - full-value property-tax rate per $10,000  
> PTRATIO - pupil-teacher ratio by town  
> **B - 1000(Bk - 0.63)^2 where Bk is the proportion of Black people by town**  
> LSTAT - % lower status of the population  
> MEDV - Median value of owner-occupied homes in $1000's  

## <a id='toc2_1_'></a>[Review data types](#toc0_)

In [4]:
X.dtypes

CRIM        float64
ZN          float64
INDUS       float64
CHAS       category
NOX         float64
RM          float64
AGE         float64
DIS         float64
RAD        category
TAX         float64
PTRATIO     float64
B           float64
LSTAT       float64
dtype: object

In [5]:
X[['CHAS', 'RAD']].head()

,CHAS,RAD
0,0,1
1,0,2
2,0,2
3,0,3
4,0,3


In [7]:
X['RAD'].value_counts(dropna=False)

RAD
24    132
5     115
4     110
3      38
6      26
2      24
8      24
1      20
7      17
Name: count, dtype: int64

In [8]:
X['CHAS'] = X['CHAS'].astype(float)
X['RAD'] = X['RAD'].astype(float)

## <a id='toc2_2_'></a>[Review duplicates](#toc0_)

In [9]:
X >> mask(X.duplicated() == True)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT


## <a id='toc2_3_'></a>[Review nulls](#toc0_)

In [10]:
X.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

In [11]:
print(X.shape)
X.head()

(506, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [12]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

## <a id='toc2_4_'></a>[Review unique categories](#toc0_)

In [13]:
X['CHAS'].value_counts()

CHAS
0.0    471
1.0     35
Name: count, dtype: int64

In [14]:
X['RAD'].value_counts()

RAD
24.0    132
5.0     115
4.0     110
3.0      38
6.0      26
2.0      24
8.0      24
1.0      20
7.0      17
Name: count, dtype: int64

## <a id='toc2_5_'></a>[Remove obvious racial bias](#toc0_)

In [15]:
X.drop('B', axis=1, inplace=True)

# <a id='toc3_'></a>[Train-test split](#toc0_)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [17]:
print(X_train.shape)
X_train.head()

(379, 12)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,9.08
172,0.13914,0.0,4.05,0.0,0.510,5.572,88.5,2.5961,5.0,296.0,16.6,14.69
80,0.04113,25.0,4.86,0.0,0.426,6.727,33.5,5.4007,4.0,281.0,19.0,5.29
46,0.18836,0.0,6.91,0.0,0.448,5.786,33.3,5.1004,3.0,233.0,17.9,14.15
318,0.40202,0.0,9.90,0.0,0.544,6.382,67.2,3.5325,4.0,304.0,18.4,10.36


In [18]:
print(X_test.shape)
X_test.head()

(127, 12)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT
307,0.04932,33.0,2.18,0.0,0.472,6.849,70.3,3.1827,7.0,222.0,18.4,7.53
343,0.02543,55.0,3.78,0.0,0.484,6.696,56.4,5.7321,5.0,370.0,17.6,7.18
47,0.22927,0.0,6.91,0.0,0.448,6.030,85.5,5.6894,3.0,233.0,17.9,18.80
67,0.05789,12.5,6.07,0.0,0.409,5.878,21.4,6.4980,4.0,345.0,18.9,8.10
362,3.67822,0.0,18.10,0.0,0.770,5.362,96.2,2.1036,24.0,666.0,20.2,10.19


# <a id='toc4_'></a>[Linear Regression](#toc0_)

In [19]:
# Create linear regression
regr = LinearRegression()

# Train the linear regression
model = regr.fit(X_train, y_train)

# Score the model
model.score(X_test, y_test)

0.7668348687111705

In [20]:
# Predict on the X_test
model.predict(X_test)

array([32.367134  , 27.89009124, 17.97334994, 21.64318065, 18.57625431,
       19.90217832, 32.33919893, 18.15456305, 24.62331067, 26.91367353,
       26.84352322, 28.54474022, 21.08444663, 26.91336328, 23.33666017,
       20.82731727, 16.54030997, 37.65484273, 30.53903473,  7.86547789,
       20.81945176, 17.209237  , 25.07919368, 24.76073365, 31.41483251,
       12.70540628, 14.58707245, 16.82797619, 35.77873377, 14.10021172,
       21.35207062, 14.03222247, 42.68839625, 17.23127594, 21.94825585,
       20.22862388, 17.33521405, 27.00470476,  9.36671691, 19.50729246,
       24.33906424, 21.07899566, 29.39978796, 16.42790236, 18.90265944,
       15.90433538, 39.36617111, 17.48228115, 27.01093338, 22.6306373 ,
       24.69216838, 24.51282601, 25.10987278, 27.25281994,  5.10488808,
       24.09397178, 10.25735884, 26.83259623, 16.7865484 , 35.43825041,
       19.36826402, 27.50509584, 15.98590338, 20.54241741, 10.93025143,
       32.09626959, 36.25691269, 21.8192897 , 24.71993812, 25.13

In [21]:
# Compare with real data
np.array(y_test)

array([28.2, 23.9, 16.6, 22. , 20.8, 23. , 27.9, 14.5, 21.5, 22.6, 23.7,
       31.2, 19.3, 19.4, 19.4, 27.9, 13.9, 50. , 24.1, 14.6, 16.2, 15.6,
       23.8, 25. , 23.5,  8.3, 13.5, 17.5, 43.1, 11.5, 24.1, 18.5, 50. ,
       12.6, 19.8, 24.5, 14.9, 36.2, 11.9, 19.1, 22.6, 20.7, 30.1, 13.3,
       14.6,  8.4, 50. , 12.7, 25. , 18.6, 29.8, 22.2, 28.7, 23.8,  8.1,
       22.2,  6.3, 22.1, 17.5, 48.3, 16.7, 26.6,  8.5, 14.5, 23.7, 37.2,
       41.7, 16.5, 21.7, 22.7, 23. , 10.5, 21.9, 21. , 20.4, 21.8, 50. ,
       22. , 23.3, 37.3, 18. , 19.2, 34.9, 13.4, 22.9, 22.5, 13. , 24.6,
       18.3, 18.1, 23.9, 50. , 13.6, 22.9, 10.9, 18.9, 22.4, 22.9, 44.8,
       21.7, 10.2, 15.4, 25.3, 23.3,  7.2, 21.2, 11.7, 27. , 29.6, 26.5,
       43.5, 23.6, 11. , 33.4, 36. , 36.4, 19. , 20.2, 34.9, 50. , 19.3,
       14.9, 26.6, 19.9, 24.8, 21.2, 23.9])

What metric do I use to check my model is doing a decent job?

In [22]:
# Get the metric manually
np.sqrt(((regr.predict(X_test) - np.array(y_test))**2).sum()/len(X_test))

4.80593195875437

# Feature engineering (not the topic of this lesson)

## <a id='toc4_1_'></a>[Review model coefficients](#toc0_)

In [23]:
model.coef_

array([-1.21641639e-01,  5.72116407e-02,  3.40296080e-02,  2.52810294e+00,
       -2.19491135e+01,  2.79629811e+00,  8.08290266e-03, -1.48764268e+00,
        2.92285787e-01, -1.07969763e-02, -9.80592515e-01, -5.74769628e-01])

In [24]:
px.histogram(x=model.coef_, y=X.columns, text_auto=True)

# <a id='toc5_'></a>[Ridge regression](#toc0_)

In [25]:
#ridge tries to control the coefficients
from sklearn.linear_model import Ridge

# Create linear regression
regr = Ridge(alpha=1.0)

# Train the linear regression
model = regr.fit(X_train, y_train)

# Score the model
model.score(X_test, y_test)

0.7720662551421119

In [26]:
# in this case improvement is... moderate
model.coef_

array([-1.17032967e-01,  5.82644199e-02, -1.28954499e-02,  2.27509521e+00,
       -1.18882103e+01,  2.88618425e+00, -2.65251779e-04, -1.34814792e+00,
        2.71180432e-01, -1.19370273e-02, -8.58988044e-01, -5.87713387e-01])

In [27]:
px.histogram(x=model.coef_, y=X.columns, text_auto=True)

In [28]:
#we can exaggerate the effect
from sklearn.linear_model import Ridge

# Create linear regression
regr = Ridge(alpha=10000)
# Train the linear regression
model = regr.fit(X_train, y_train)
print(model.score(X_test, y_test)) # The model becomes much worse
model.coef_

0.503472854700465


array([-0.08488274,  0.05749313, -0.04967859,  0.01239866, -0.00093485,
        0.06982266,  0.00487248, -0.08037034,  0.08732858, -0.01123352,
       -0.13297838, -0.44741359])

In [29]:
px.histogram(x=model.coef_, y=X.columns, text_auto=True)

In [30]:
# Create linear regression
regr = Ridge(alpha=0.01)

# Train the linear regression
model = regr.fit(X_train, y_train)

# Score the model
model.score(X_test, y_test)

0.7669914734642315

In [31]:
px.histogram(x=model.coef_, y=X.columns, text_auto=True)

## <a id='toc5_1_'></a>[Lasso regression](#toc0_)

In [35]:
from sklearn.linear_model import Lasso

# Create linear regression
regr = Lasso(alpha=0.1)

# Train the linear regression
model = regr.fit(X_train, y_train)

# Score the model
model.score(X_test, y_test)

0.7609097744779654

In [36]:
#not worth it in this case, since it is underfitting, but see what happens to the coefficients
model.coef_

array([-0.11057616,  0.05960726, -0.05489593,  0.82240637, -0.        ,
        2.76157266, -0.00568496, -1.13089696,  0.25461411, -0.01391173,
       -0.72707696, -0.62370184])

In [37]:
px.histogram(x=model.coef_, y=X.columns, text_auto=True)

## <a id='toc5_2_'></a>[Retry regression without 'useless' features](#toc0_)

In [ ]:
X_lasso = X.drop(['RM', 'NOX', 'CHAS', 'INDUS'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_lasso, y, test_size=0.25, random_state=1)

In [ ]:
# Create linear regression
regr = Ridge()

# Train the linear regression
model = regr.fit(X_train, y_train)

# Score the model
model.score(X_test, y_test)

# <a id='toc6_'></a>[Decision Tree Regression](#toc0_)

In [ ]:
# Import libraries
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Create decision tree classifer object
regr = DecisionTreeRegressor()

# Train regression tree model
model = regr.fit(X_train, y_train)

# Score the model
regr.score(X_test, y_test)

In [ ]:
y_pred = regr.predict(X_test)
y_pred

In [ ]:
X_train.head()

In [ ]:
from sklearn import tree
# Expect this to take some time
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(model, 
                   feature_names=model.feature_names_in_, 
                   filled=True)

In [ ]:
#let's look at the decision tree
from sklearn.tree import export_text

r = export_text(regr, feature_names=list(X_train.columns))
print(r)

# <a id='toc7_'></a>[KNN Regression](#toc0_)

In [ ]:
# Import libraries
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 1)

In [ ]:
# Since the KNN algorithm is distance-based, we also include a scaling step before training
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# create knn model
knnr = KNeighborsRegressor(n_neighbors=3, weights='distance')

# Train the knn model
model = knnr.fit(X_train, y_train)

# Predict and score the model
knnr.score(X_test, y_test)

In [ ]:
y_pred = knnr.predict(X_test)
y_pred

In [ ]:
np.array(y_test)

# <a id='toc8_'></a>[Regression evaluation metrics](#toc0_)

**Similarities:** [$^{[3]}$](https://archive.is/20230531192343/https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d)
> - Both MAE and RMSE express average model prediction error in units of the variable of interest. 
> - Both metrics can range from 0 to ∞ and are indifferent to the direction of errors. 
> - They are negatively-oriented scores, which means lower values are better.

**Differences:** [$^{[3]}$](https://archive.is/20230531192343/https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d)
> - MAE is a measure of **average error**, whereas RMSE is a measure of the **variance of the error frequency distribution**

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, y_pred)

In [ ]:
abs(y_test-y_pred).mean()

## <a id='toc8_1_'></a>[R/MSE](#toc0_)

In [ ]:
from sklearn.metrics import mean_squared_error

display(mean_squared_error(y_test, y_pred))
display(np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
np.sqrt(((y_test-y_pred)**2).mean())

## <a id='toc8_2_'></a>[$R^2$](#toc0_)

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

# <a id='toc9_'></a>[References](#toc0_)

[1] [Revisiting the Boston Housing Dataset, Fairlearn](https://fairlearn.org/main/user_guide/datasets/boston_housing_data.html)  
[2] [Boston Housing Dataset, Kaggle](https://www.kaggle.com/code/prasadperera/the-boston-housing-dataset)  
[3] [MAE and RMSE — Which Metric is Better?, Medium](https://archive.is/20230531192343/https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d)

# <a id='toc10_'></a>[Acknowledgements](#toc0_)

Thank you, David Henriques, for your awesome lesson structure and content!